# Kalman Filter for TurtleBot Localization

**Paper:** Kalman (1960) - A new approach to linear filtering and prediction problems

**Implementation:** TurtleBot localization using Kalman Filter

This notebook demonstrates the implementation of a Kalman Filter for TurtleBot position estimation using odometry and sensor data.

## Download

You can download this notebook and run it locally:
- Click `File` → `Download as` → `Notebook (.ipynb)`
- Or use the download button in the top-right corner

## Requirements

```bash
pip install numpy matplotlib pykal
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.utilities.estimators import KF

# Enable inline plotting
%matplotlib inline

## TurtleBot Motion Model

The TurtleBot state is represented as:
$$
\mathbf{x} = \begin{bmatrix} x \\ y \\ \theta \end{bmatrix}
$$

Where $(x, y)$ is position and $\theta$ is heading angle.

The discrete-time dynamics with velocity commands $(v, \omega)$ are:
$$
\mathbf{x}_{k+1} = f(\mathbf{x}_k, \mathbf{u}_k) = \begin{bmatrix}
x_k + v_k \cos(\theta_k) \Delta t \\
y_k + v_k \sin(\theta_k) \Delta t \\
\theta_k + \omega_k \Delta t
\end{bmatrix}
$$

In [ ]:
# Simulation parameters
dt = 0.1  # time step
T = 10.0  # total time
steps = int(T / dt)

# Define the motion model
def turtlebot_dynamics(xhat, uk):
    """
    TurtleBot motion model.
    
    Args:
        xhat: state [x, y, theta]
        uk: control [v, omega]
    
    Returns:
        Next state prediction
    """
    x, y, theta = xhat
    v, omega = uk
    
    x_next = x + v * np.cos(theta) * dt
    y_next = y + v * np.sin(theta) * dt
    theta_next = theta + omega * dt
    
    return np.array([x_next, y_next, theta_next])

# Observation model (GPS-like sensor)
def observation_model(xhat):
    """Observe position only (not heading)."""
    return xhat[:2]  # Return [x, y]

print("✓ Models defined")

## Kalman Filter Setup

We'll use the pykal implementation of the Kalman Filter to estimate the TurtleBot's position.

In [ ]:
# Initial state
x0 = np.array([0.0, 0.0, 0.0])  # Start at origin, facing east

# Kalman filter initial estimate
xhat0 = x0 + np.random.randn(3) * 0.1  # Add some initial uncertainty
P0 = np.eye(3) * 0.5  # Initial covariance

# Process noise covariance
Q = np.diag([0.01, 0.01, 0.05])  # Noise in [x, y, theta]

# Measurement noise covariance  
R = np.diag([0.1, 0.1])  # Noise in [x, y] observations

# Jacobians for linearization
def F_jacobian(xhat, uk):
    """Jacobian of dynamics w.r.t. state."""
    _, _, theta = xhat
    v, omega = uk
    
    return np.array([
        [1, 0, -v * np.sin(theta) * dt],
        [0, 1,  v * np.cos(theta) * dt],
        [0, 0, 1]
    ])

def H_jacobian(xhat):
    """Jacobian of observation w.r.t. state."""
    return np.array([
        [1, 0, 0],
        [0, 1, 0]
    ])

print("✓ Kalman Filter configured")

## Simulation

Let's simulate the TurtleBot driving in a circular path and track it with the Kalman Filter.

In [ ]:
# Control input (circular motion)
v = 1.0  # Forward velocity
omega = 0.5  # Angular velocity

# Storage for results
true_states = [x0]
estimates = [xhat0]
observations = []

# Current state and estimate
x_true = x0.copy()
xhat_P = (xhat0, P0)

# Run simulation
for k in range(steps):
    # True dynamics with process noise
    uk = np.array([v, omega])
    x_true = turtlebot_dynamics(x_true, uk) + np.random.multivariate_normal([0,0,0], Q)
    
    # Noisy observation
    yk = observation_model(x_true) + np.random.multivariate_normal([0,0], R)
    
    # Kalman filter update
    param_dict = {
        'xhat_P': xhat_P,
        'yk': yk,
        'f': turtlebot_dynamics,
        'f_params': {'xhat': xhat_P[0], 'uk': uk},
        'F': F_jacobian,
        'F_params': {'xhat': xhat_P[0], 'uk': uk},
        'Q': lambda: Q,
        'Q_params': {},
        'h': observation_model,
        'h_params': {'xhat': xhat_P[0]},
        'H': H_jacobian,
        'H_params': {'xhat': xhat_P[0]},
        'R': lambda: R,
        'R_params': {}
    }
    
    xhat_P = KF.f(**param_dict)
    
    # Store results
    true_states.append(x_true.copy())
    estimates.append(xhat_P[0].copy())
    observations.append(yk.copy())

true_states = np.array(true_states)
estimates = np.array(estimates)
observations = np.array(observations)

print(f"✓ Simulation complete ({steps} steps)")

## Results Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot trajectory
ax1.plot(true_states[:, 0], true_states[:, 1], 'b-', label='True Path', linewidth=2)
ax1.plot(estimates[:, 0], estimates[:, 1], 'r--', label='KF Estimate', linewidth=2)
ax1.scatter(observations[:, 0], observations[:, 1], c='gray', alpha=0.3, s=10, label='Noisy Obs')
ax1.set_xlabel('X Position (m)')
ax1.set_ylabel('Y Position (m)')
ax1.set_title('TurtleBot Trajectory with Kalman Filter')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axis('equal')

# Plot estimation error
error = np.linalg.norm(true_states[:, :2] - estimates[:, :2], axis=1)
time = np.arange(len(error)) * dt
ax2.plot(time, error, 'r-', linewidth=2)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Position Error (m)')
ax2.set_title('Kalman Filter Estimation Error')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average position error: {np.mean(error):.4f} m")
print(f"Max position error: {np.max(error):.4f} m")

## Conclusion

This notebook demonstrated:
1. Implementing a Kalman Filter for TurtleBot localization
2. Using the pykal framework for state estimation
3. Visualizing the filter performance

The Kalman Filter successfully tracks the TurtleBot's position despite noisy observations.

## Next Steps

- Try different motion patterns (straight line, figure-8, etc.)
- Adjust noise parameters Q and R
- Extend to include velocity estimation
- Test with real TurtleBot data

## References

- Kalman, R.E. (1960). "A new approach to linear filtering and prediction problems"
- pykal documentation: https://pykal.readthedocs.io